In [1]:
import pandas as pd
import argparse
import os
import sklearn.metrics
from aggregate_entity_results import aggregate_results
from utils import generate_explanation_prediction
import json

In [2]:
size = 1.0
version_str = "lv_v1_tv_v1"
agg_method = "expert"
model_name = "gpt-3.5-turbo"
# few_shot_selection = "label_diversity_similarity"
few_shot_selection="semantic_similarity"
# verify_few_shot_selection = "label_diversity_similarity"
verify_few_shot_selection="semantic_similarity"
temperature = 0.1
verified = True
explanation_column = "two_step"
check_mode = "strict"
CoT = True
verify_CoT = False
use_current_explanation = False
use_ground_explanation = False
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.traintestfortest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-' + agg_method
label_symbol = "^^^^"
text_column = "context"
result_dir = '/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-' + agg_method
archive = False
if archive:
    output_dir = os.path.join(output_dir, 'archive', str(temperature), str(size), version_str)
    result_dir = os.path.join(result_dir, 'archive', str(temperature), str(size), version_str)
else:
    output_dir = os.path.join(output_dir, str(temperature), str(size), version_str)
    result_dir = os.path.join(result_dir, str(temperature), str(size), version_str)

In [3]:
if verified:
    final_name = '_aggregated_final_COT_' if verify_CoT else '_aggregated_final_'
    if few_shot_selection != verify_few_shot_selection:
            final_name = final_name + verify_few_shot_selection + '_'
    if CoT:
        agg_output_name = os.path.split(model_name)[-1] + '_' + check_mode + '_' + few_shot_selection + '_COT_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + final_name + os.path.split(data_path)[-1]
    else:
        agg_output_name = os.path.split(model_name)[-1] + '_' + check_mode + '_' + few_shot_selection + '_' + final_name + os.path.split(data_path)[-1]
else:
    if CoT:
        agg_output_name = os.path.split(model_name)[-1] + '_' + check_mode + '_' + few_shot_selection + '_COT_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_aggregated_' + os.path.split(data_path)[-1]
    else:
        agg_output_name = os.path.split(model_name)[-1] + '_' + check_mode + '_' + few_shot_selection + '_aggregated_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, agg_output_name)
print(output_path)
output_df = pd.read_csv(output_path)

/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-expert/0.1/1.0/lv_v1_tv_v1/gpt-3.5-turbo_strict_semantic_similarity_COT_two_step_cur_False_ground_False_aggregated_final_expert.traintestfortest.csv


In [4]:
output_df.columns

Index(['id', 'tweet', 'tweet_tokens', 'entity_label', 'related_label',
       'sentence_class', 'relevant_entity_label',
       'relevance_entity_class_label', 'real_relevant_entity_label',
       'IndividualLabels', 'CategorizedLabels', 'Food_answer',
       'Location_answer', 'Symptom_answer', 'Keyword_answer', 'has_Food',
       'has_Location', 'has_Symptom', 'has_Keyword', 'has_all', 'context',
       'sentence_explanation', 'word_explanation',
       'CategorizedLabels_prediction', 'sentence_explanation_prediction'],
      dtype='object')

In [5]:
output_df['has_Food'].value_counts()

has_Food
no     3338
yes     372
Name: count, dtype: int64

In [6]:
output_df['has_all'].value_counts()

has_all
no     2485
yes    1225
Name: count, dtype: int64

In [7]:
word_pred = output_df['sentence_explanation_prediction'].str.startswith('The word')

In [8]:
word_label = output_df[['CategorizedLabels', 'sentence_class']].apply(lambda x: generate_explanation_prediction(x['CategorizedLabels'], x['sentence_class']), axis=1).str.startswith('The word')

In [9]:
def count_entities(label_dict):
    label_dict = json.loads(label_dict)
    cnt = 0
    for key, val in label_dict.items():
        cnt += len(val)
    return cnt

In [10]:
output_df['CategorizedLabels'].apply(count_entities).sum()

2648

In [11]:
output_df['CategorizedLabels_prediction'].apply(count_entities).sum()

4709

In [12]:
label = output_df['sentence_class'] == 'Yes'

In [13]:
sklearn.metrics.accuracy_score(label, word_pred)

0.6622641509433962

In [14]:
sklearn.metrics.f1_score(label, word_pred)

0.6491178941472977

In [15]:
sklearn.metrics.precision_score(label, word_pred)

0.4940323955669224

In [16]:
sklearn.metrics.recall_score(label, word_pred)

0.9461224489795919

In [17]:
sklearn.metrics.accuracy_score(label, word_label)

1.0

In [18]:
sklearn.metrics.f1_score(label, word_label)

1.0